# 美しく姿を変える 「ホログラム菓子」を作る 台所は工場、調理は製造、砂糖で作る光学レンズ素子

## 「型枠を作るための“原型”」の 3D モデルを出力する Python コード

In [2]:
import math
from solid import * # pip install solidpython

def make_sphere_at_pos(size_d, xyz): # 直径, 場所[x,y,z]
    return translate(xyz)(sphere(d=size_d, segments=4)) # 4は形状の滑らかさ

def make_cube_at_pos(size_xyz, xyz):
    return translate(xyz)(cube(size_xyz, center=True))

x_num = 10#30 # 60 for 1.8mm, 40 for 3mm
y_num = 10#30 # 60
r = 1.5 # 0.9 # 1.5 # 球の半径（mm）
d = 2 * r # 球の直径でもあり、広い方向のspaceでもある
narrow_space=math.sqrt(3)*r # 狭い方向のspace
base_thickness = 4  # 厚み(mm)
base_height = 5     # 高さ(mm)
base_over_height=10 # 余分な高さ（mm）
edge_length = 3     # 周囲幅(mm)
base_x_size = (x_num) * narrow_space+2*edge_length
base_y_size = (x_num) * d + 2*edge_length
base_x_outer_size = base_x_size + 2*base_thickness
base_y_outer_size = base_y_size + 2*base_thickness
silicone_thickness = 4 # mm

# 直方体を作る
mold_base = make_cube_at_pos([base_x_outer_size, 
                              base_y_outer_size,
                              base_height+base_thickness+base_over_height],
                             [base_x_size/2, 
                              base_y_size/2, 
                              (base_height+base_thickness+base_over_height)/2])
# 直方体を削る
mold_base = mold_base - make_cube_at_pos(
                             [base_x_size, 
                              base_y_size,
                              base_over_height+base_thickness],
                             [base_x_size/2,
                              base_y_size/2, 
                              base_height+base_thickness+(base_over_height+base_thickness)/2])
# 周囲の「お堀」部分
mold_base = mold_base - make_cube_at_pos(
                             [base_x_size, 
                              silicone_thickness,
                              base_height+base_over_height+base_thickness],
                             [base_x_size/2,
                              silicone_thickness/2, 
                              base_thickness+(base_height+base_over_height+base_thickness)/2])
mold_base = mold_base - make_cube_at_pos(
                             [silicone_thickness, 
                              base_y_size,
                              base_height+base_over_height+base_thickness],
                             [silicone_thickness/2,
                              base_y_size/2, 
                              base_thickness+(base_height+base_over_height+base_thickness)/2])
mold_base = mold_base - make_cube_at_pos(
                             [base_x_size, 
                              silicone_thickness,
                              base_height+base_over_height+base_thickness],
                             [base_x_size/2,
                              base_y_size-silicone_thickness/2, 
                              base_thickness+(base_height+base_over_height+base_thickness)/2])
mold_base = mold_base - make_cube_at_pos(
                             [silicone_thickness, 
                              base_y_size,
                              base_height+base_over_height+base_thickness],
                             [base_x_size-silicone_thickness/2,
                              base_y_size/2, 
                              base_thickness+(base_height+base_over_height+base_thickness)/2])

# 球形部分を除去（3Dプリンタだけで型を作るなら足すところだが、ビーズ球を使うため、抜く）
spheres = []
for y in range(y_num):
    Y=y*d+r+edge_length
    Z=base_height+base_thickness 
    for x in range(x_num):
        X=x*narrow_space+r+edge_length
        if x%2 == 0:
            spheres.append( make_sphere_at_pos(d,[X,Y,Z])   )
        else:
            spheres.append( make_sphere_at_pos(d,[X,Y+r,Z]) )
# 最終的に生成された形状
mold_base = difference()(mold_base,spheres)
# .scad ファイルに出力する
scad_render_to_file(mold_base, "lensMold.scad")

## 3Dプリンタで 出力できる .stlファイルに変換する

In [3]:
import subprocess

# OpenSCADを使って、.scad ファイルをSTLファイルに変換する
openscad = '/Applications/OpenSCAD.app/Contents/MacOS/OpenSCAD'
subprocess.run([openscad, 'lensMold.scad','-o lensMold.stl','-q'])

'/Users/jun/source/SD/SD202312_hologram/lensMold.scad'

## 方向(9 方向)に応じて、 異なる色が見える画像模様を作り出す Python コード

In [ ]:
import math

x_num = 30 # 
y_num = 30 # 
r = 1.5    # 球の半径（mm）
d = 2 * r  # 球の直径でもあり、  広い方向のspace ( in mm)
narrow_space = math.sqrt(3)*r # 狭い方向のspace ( in mm)

# ピクセル数を計算する（プリント時には、現物合わせで変倍する）
pixels_x = int(narrow_space * x_num / 25.4 * 600)  # in pixel
pixels_y = int(d * y_num / 25.4 * 600)             # in pixel
# セルあたりピクセル数を計算する
pixels_of_cell_in_y = int(pixels_y / y_num)
pixels_of_cell_in_x = int(pixels_x / x_num)

import numpy as np
import cv2

# 変角画像を読み込む
files = ["001.png","002.png","003.png","004.png","005.png",
         "006.png","007.png","008.png","009.png"]
images = [cv2.resize(cv2.imread("001.png"), dsize=(x_num, y_num)) for f in files]

import matplotlib.pyplot as plt

# 変角画像を表示
for image in images:
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

# テクスチャ画像バッファを生成する
background_image2 = np.zeros([pixels_y, pixels_x, 3], dtype = 'uint8')

# テクスチャ画像バッファに、レンズアレイ毎に「変角画像」を貼り付ける
for y in range(y_num):
    for x in range(x_num):
        # 張り込む「変角」画像を生成する
        patch = np.zeros( [ round(pixels_of_cell_in_y),
                            round(pixels_of_cell_in_x), 3], 
                           dtype = 'uint8')
        for y1 in range(3):
            for x1 in range(3):
                patch[ 
            round(y1*pixels_of_cell_in_y/3):round((y1+1)*pixels_of_cell_in_y/3),
            round(x1*pixels_of_cell_in_x/3):round((x1+1)*pixels_of_cell_in_x/3)] = \
                images[3*y1+x1][y,x]
        # 「変角」画像をテクスチャ画像バッファに貼り付ける
        sy = round(y*pixels_of_cell_in_y)
        if y%2 is 0: # 14列の行
            if x < (x_num-1):
                sx = round(x*pixels_of_cell_in_x+pixels_of_cell_in_x/2)
                background_image2[sy:round(sy+pixels_of_cell_in_y), sx: \
                                  round(sx+pixels_of_cell_in_x)]= patch
        else:        # 15列の行
            sx = round(x*pixels_of_cell_in_x)
            background_image2[sy:round(sy+pixels_of_cell_in_y), \ 
                              sx:round(sx+pixels_of_cell_in_x)] = patch
# テクスチャ画像を保存する
cv2.imwrite('patch_30x30.png', background_image2)
# テクスチャ画像を表示
plt.figure(figsize=(4.217,4.217))
plt.imshow(cv2.cvtColor(background_image2, cv2.COLOR_BGR2RGB))